In [3]:
import fasttext
import os
import pandas as pd
import numpy as np

In [4]:
from gensim import models
from datetime import date

In [5]:
today = date.today().strftime('%y%m%d')
print(today)

191118


In [8]:
WIKI_KO_DATA = './data/cc.ko.300.bin.gz'
KO_MODEL_PATH = f'./model/{today}_ko_model'

In [7]:
if not os.path.exists('./model'):
    os.makedirs('./model')

### Gensim FastText
- cc.ko.300.bin.gz : [Fast Text에서 제공하는 위키백과 한국어 데이터](https://fasttext.cc/docs/en/crawl-vectors.html)

In [22]:
ko_model = models.fasttext.load_facebook_model(WIKI_KO_DATA)

NameError: name 'm_fasttext' is not defined

In [26]:
ko_model.save(KO_MODEL_PATH)

In [11]:
ko_model = models.fasttext.load_facebook_model(KO_MODEL_PATH)

NotImplementedError: Supervised fastText models are not supported

In [23]:
for w, sim in ko_model.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')

/Users/nero/.virtualenvs/dc/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


Python: 0.565061628818512
자이썬: 0.5624369382858276
레일스: 0.5598082542419434
파이썬을: 0.5595801472663879
언어용: 0.5288202166557312
파이썬의: 0.5250024795532227
프로그래밍: 0.5225088000297546
wxPython: 0.5222088098526001
파이썬이나: 0.5201171636581421
함수형: 0.5187377333641052


In [33]:
DATA_PATH = './data/191118_data.tsv'

In [36]:
data = pd.read_csv(DATA_PATH, sep='\t', usecols=['label', 'sentence'])
data.sample(5)

,label,sentence
4697,0,git https leetcode 236 lowest common ancestor ...
3130,1,reinforcement learning papers accepted to neur...
6624,1,고정밀 gnss와 ros 연동하기 이 글은 고정밀 gnssglobal navigat...
7862,1,binary degreeor binary radian brad이진 각도 to deg...
6291,1,rxjs를 시작하기전에 rxjsrxjs 시리즈 포스팅은 quick start rxj...


In [65]:
data.sentence

0       todo 리스트를 1달간 사용해본 후기 왜 todo 리스트를 사용했나요 위 질문에 ...
1       양평 블룸비스타 콩순이 패밀리룸이 궁금하신분에게 드리는 후기 늦은 여름 휴가로 양평...
2       javascript js 자바스크립트 문자열 메소드 javascript 문자열 메소...
3       우리 시장에는 어떤 교육이 필요할까요 1 시장적합형 교육과 기술적합형 교육 며칠전 ...
4       저 많은 국민은행 앱들은 대체 다 어디다 쓰는 걸까 대략 정리 구글플래이 스토어에서...
                              ...                        
9995    nginx rest kubernetes 164 kubernetes nginx ing...
9996    git https javascript html web ui modal design ...
9997    spring boot spring spring boot오류 처리에 대해 서론오류 처...
9998    http3 http3 httpoverquic quic은 quick udp inter...
9999    spring kotlin kotlin spring boot security thym...
Name: sentence, Length: 10000, dtype: object

In [68]:
def sentenceToVector(sentence):
    words = sentence.split(' ')
    words_num = len(words)
    vector = np.zeros(300) # ko_model shape가 300
    for word in words:
        vector += ko_model[word]
    return vector/words_num
data['vector'] = data.sentence.apply(sentenceToVector)

/Users/nero/.virtualenvs/dc/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [82]:
a = 1 # [0, 1]
b = 0 # [1, 0]
print(([0]*2)[a] = 1)

SyntaxError: keyword can't be an expression (<ipython-input-82-316d74312ba3>, line 3)

In [85]:
def numberToOneHot(num):
    oneHot = [0]*2
    oneHot[num] = 1
    return oneHot

data['oneHot'] = data.label.apply(numberToOneHot)

In [86]:
data.head(3)

,label,sentence,vector,oneHot
0,1,todo 리스트를 1달간 사용해본 후기 왜 todo 리스트를 사용했나요 위 질문에 ...,"[0.013710441264420491, 0.0032995063650540692, ...","[0, 1]"
1,0,양평 블룸비스타 콩순이 패밀리룸이 궁금하신분에게 드리는 후기 늦은 여름 휴가로 양평...,"[0.00013062362416590355, -0.032540909874690126...","[1, 0]"
2,1,javascript js 자바스크립트 문자열 메소드 javascript 문자열 메소...,"[-0.04456514536979653, 0.00961365589007203, 0....","[0, 1]"


In [105]:
data.vector[0].shape

(300,)

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.vector, data.oneHot, test_size=0.33, random_state=321)

In [114]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(34576347859, 300)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2)